In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import PyPizza
import os

# --- CONFIGURAÇÕES VISUAIS (Dark & Clean) ---
COLOR_BG = "#121212"       # Preto Fosco (Fundo)
COLOR_TEXT = "#E0E0E0"     # Branco Gelo
COLOR_SLICE = "#1e1e1e"    # Fundo da Fatia (Cinza escuro)
COLOR_MAIN = "#00d4ff"     # Azul Neon (Destaque)

def load_data():
    path = "data/processed/final_ranking_table.csv"
    if not os.path.exists(path):
        raise FileNotFoundError(f"Arquivo {path} não existe.")
    return pd.read_csv(path)

def create_radar(player_row):
    """Gera o card do jogador."""
    rank = int(player_row['Rank'])
    name = player_row['Player']
    team = player_row['Team']
    
    # Valores Normalizados (0-100)
    values = [
        round(player_row['Score_Shot_Stopping'], 1),
        round(player_row['Score_Distribution'], 1),
        round(player_row['Score_Aerial'], 1)
    ]
    
    params = [
        "Shot Stopping\n(Volume + Eficiência)", 
        "Distribuição\n(Saída de Bola)", 
        "Controle Aéreo\n(Cruzamentos)"
    ]

    # Configuração do Gráfico
    baker = PyPizza(
        params=params,
        background_color=COLOR_BG,
        straight_line_color=COLOR_BG,
        straight_line_lw=1,
        last_circle_lw=0,
        other_circle_lw=0,
        inner_circle_size=20
    )

    fig, ax = baker.make_pizza(
        values,
        figsize=(8, 8),
        param_location=110,
        kwargs_slices=dict(facecolor=COLOR_MAIN, edgecolor=COLOR_BG, zorder=2, linewidth=1),
        kwargs_params=dict(color=COLOR_TEXT, fontsize=11, va="center", fontfamily="sans-serif"),
        kwargs_values=dict(
            color=COLOR_BG, fontsize=11, zorder=3,
            bbox=dict(edgecolor=COLOR_MAIN, facecolor=COLOR_MAIN, boxstyle="round,pad=0.2", lw=1)
        )
    )

    # Títulos
    fig.text(0.515, 0.97, f"#{rank} {name}", size=22, ha="center", color=COLOR_TEXT, weight="bold")
    
    score = round(player_row['Final_Score'], 1)
    fig.text(0.515, 0.93, f"{team} | Score Final: {score}", size=14, ha="center", color=COLOR_MAIN)
    
    # Rodapé Técnico
    note = "Métrica: Média Ponderada (Eficiência p90 + Ratio de Prevenção)"
    fig.text(0.99, 0.02, note, color=COLOR_TEXT, ha="right", fontsize=8, alpha=0.5)

    # Salvar
    filename = f"assets/rank_{rank:02d}_{name.replace(' ', '_')}.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight', facecolor=COLOR_BG)
    plt.close()
    print(f"   📸 Gerado: {filename}")

def main():
    print("🎨 Gerando Galeria de Goleiros...")
    os.makedirs("assets", exist_ok=True)
    
    df = load_data()
    
    # 1. Top 5 Geral
    for i in range(5):
        if i < len(df): create_radar(df.iloc[i])
            
    # 2. Goleiros de Interesse (Big Six / Famosos)
    targets = ["Alisson", "Ederson", "Raya", "Onana", "Vicario", "Martinez", "Donnarumma"]
    
    for name in targets:
        # Acha o jogador no dataframe (ignora case)
        matches = df[df['Player'].str.contains(name, case=False, na=False)]
        if not matches.empty:
            row = matches.iloc[0]
            # Só gera se não for Top 5 (para não duplicar)
            if row['Rank'] > 5:
                create_radar(row)

if __name__ == "__main__":
    try:
        main()
        print("\n✅ Imagens salvas na pasta 'assets/'")
    except Exception as e:
        print(f"❌ Erro: {e}")

🎨 Gerando Galeria de Goleiros...
   📸 Gerado: assets/rank_01_Robin_Roefs.png
   📸 Gerado: assets/rank_02_Robert_Sánchez.png
   📸 Gerado: assets/rank_03_Dean_Henderson.png
   📸 Gerado: assets/rank_04_Emiliano_Martínez.png
   📸 Gerado: assets/rank_05_Jordan_Pickford.png
   📸 Gerado: assets/rank_11_Alisson.png
   📸 Gerado: assets/rank_13_David_Raya.png
   📸 Gerado: assets/rank_06_Guglielmo_Vicario.png
   📸 Gerado: assets/rank_09_Gianluigi_Donnarumma.png

✅ Imagens salvas na pasta 'assets/'
